In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import time
from glob import glob
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.io import read_image

%pylab inline

c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Populating the interactive namespace from numpy and matplotlib


In [25]:
DEVICE = 'cuda'

folder = '../data/'
files = glob(f"{folder}*/*.jpg", recursive = True)
train_f, test_f = train_test_split(files, test_size=0.3, random_state=23, shuffle=True)

In [3]:
def plot_img(path):
    img = mpimg.imread(path)
    imgplot = plt.imshow(img)
    plt.show()

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
                         std  = [ 0.229, 0.224, 0.225 ]),
    ])

target_map = {
    'apple_season': 0,
    'lemon': 1,
    'persimmon': 2,
    'qiwi': 3,
    'tomato': 4
}
target_transform = lambda x: target_map[x]

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, files, transform=None, target_transform=None, test=False):
        self.files = files
        self.transform = transform
        self.target_transform = target_transform
        self.test = test
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        image = Image.open(img_path)
        label = img_path.split('\\')[1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [86]:
tr_set = CustomImageDataset(train_f, transform=transform, target_transform=target_transform)
t_set = CustomImageDataset(test_f, transform=transform, target_transform=target_transform, test=True)
tr_dl = DataLoader(tr_set, batch_size=128, shuffle=True)
t_dl = DataLoader(t_set, batch_size=128, shuffle=False)

In [87]:
from torchvision.models import mobilenet_v2
net = mobilenet_v2(pretrained=True)
net.classifier[1] = torch.nn.Linear(1280, len(target_map), bias=True)
net.classifier = torch.nn.Sequential(
    *net.classifier,
    torch.nn.Softmax(),
)

for param in net.parameters():
    param.requires_grad = False
for i in (17, 18):
    for param in net.features[i].parameters():
        param.requires_grad = True
for param in net.classifier.parameters():
    param.requires_grad = True
net = net.cuda()

c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [88]:
def fit_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc


def eval_epoch(model, val_loader, criterion, metric):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    val_preds = np.array([])
    val_labels = []
    
    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # preds = torch.argmax(outputs, 1)
        outputs = outputs.cpu()
        labels = labels.cpu()    
        if len(val_preds):
            val_preds = np.concatenate((val_preds, outputs), 0)
        else:
            val_preds = outputs

        val_labels += labels.tolist()      
        running_loss += loss.item() * inputs.size(0)
        running_corrects += (outputs.argmax(1) == labels.data).sum()
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.float() / processed_size
    val_auc = metric(val_labels, val_preds)
    return val_loss, val_acc, val_auc

In [89]:
def train(train_loader, val_loader, model, epochs, opt, scheduler, criterion, metric=roc_auc_score):
    start_time = time.time()
    history = []
    history_auc = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f} val_auc {v_auc:0.4f}"
    
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)
            
            val_loss, val_acc, val_auc = eval_epoch(model, val_loader, criterion, metric)
            history.append((train_loss, train_acc, val_loss, val_acc))
            history_auc.append(val_auc)
            scheduler.step(val_acc)
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc, v_auc=val_auc))
    
    end_time = time.time()
    
    print('total time:',end_time-start_time)
    print('average time per epoch:',(end_time-start_time)/epochs)
    return history, history_auc

In [90]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,8], gamma=0.2)
metric = lambda x, y: roc_auc_score(x, y, multi_class='ovo')

In [91]:
history, history_auc = train(tr_dl, t_dl, model=net, epochs=2, opt=optimizer, scheduler=scheduler, criterion=loss, metric=metric)

epoch:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss 0.964608170131593


c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\torch\optim\lr_scheduler.py:163: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Epoch 001 train_loss: 0.9646     val_loss 0.9584 train_acc 0.9414 val_acc 0.9450 val_auc 0.9974


c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss 0.9404161695157135

Epoch 002 train_loss: 0.9404     val_loss 0.9189 train_acc 0.9651 val_acc 0.9854 val_auc 0.9997
total time: 885.2681188583374
average time per epoch: 442.6340594291687


c:\Users\abapo\anaconda3\envs\ml\lib\site-packages\torch\optim\lr_scheduler.py:163: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


In [92]:
history_auc

[0.9973909122909236, 0.9996611356933218]

In [73]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
        for inputs, _ in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = torch.cat(logits).cpu().numpy()
    return probs

In [94]:
torch.save(net.state_dict(), '../models/mobilenet_2ep.pkl')

: 